# Housing Inventory Export

This notebook imports the cleaned (by "Housing Inventory Import.ipynb") SF Housing Inventory data from the SF open data portal. This is an alternative to development activity from the SF Development Pipeline, which I used for my CP 255 final project. 

In [80]:
#import pandas
import pandas as pd
import numpy as np
import re as re
import os
import requests  # library for accessing content from web URLs
import pprint  # library for making Python data structures readable
pp = pprint.PrettyPrinter()
import json    # library for working with JSON-formatted text strings
import time
from geopy.geocoders import GoogleV3 #google geocoding
pd.set_option('display.max_row', 1000) #sets max rows to be seen
pd.set_option('display.max_columns', 50) #Set max column width to 50

# Export Data to GeoJson

In [81]:
#set file path
root = '/Users/briangoggin/Dropbox/CP 255/SF Development Project'

In [82]:
#import cleaned data
import_path = root+"/Code/Maps/Examples Under Construction"
full_df = pd.read_csv(import_path+"/intermediate_HI.csv")

In [83]:
#Define function to create categories for javascript maps. Each category will be separate dot color
def cats(value):
    if value['net_units'] <0: 
        field = 0
        
    elif (value['net_units'] >=0) & (value['net_units']<=15):
        field = 1
        
    elif (value['net_units'] >=16) & (value['net_units']<=30):
        field = 2
        
    elif (value['net_units'] >=31) & (value['net_units']<=50):
        field = 3
        
    elif (value['net_units'] >=51) & (value['net_units']<=100):
        field = 4  
        
    else: 
        field = 5
        
    return field


full_df['unitcat'] = full_df.apply(cats, axis = 1)

In [84]:
#get rid of empty latlngs (temporary until I fix geocoding issues)
full_df = full_df[full_df['lat'].notnull()]

In [85]:
#write function to export point data
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [86]:
# Omitting "Latest Date" variable here because it does not work with json output. Not needed for now anyway
varlist = ['address', 'block', 'lot', 'status', 'units', 
           'net_units', 'affordable_units', 'zone', 'year', 'unitcat']

geojson = df_to_geojson(full_df, varlist, 'lat', 'lon')

In [87]:
# save the geojson result to a file
output_path = root+'/Code/Maps/Examples Under Construction/Housing Inventory'
output_filename = output_path+'/data.js'
with open(output_filename, 'w') as output_file:
    output_file.write('var dataset = {};'.format(json.dumps(geojson, indent=4)))

In [88]:
full_df[full_df['address'] == '125 TRENTON ST, San Francisco, CA']

,Unnamed: 0,address,affordable_units,block,lat,latest_date,latlng,lon,lot,net_units,status,units,year,zone,unitcat
371,19,"125 TRENTON ST, San Francisco, CA",1,179,37.796159,03-Sep-14,"37.7961585,-122.4093167,OK",-122.409317,30,1.0,OK,5.0,2014,CRNC,1
600,19,"125 TRENTON ST, San Francisco, CA",NaN,1127,37.796159,2013-05-15T00:00:00.000,"37.7961585,-122.4093167,OK",-122.409317,64,19.0,OK,19.0,2013,RC-4,2
818,19,"125 TRENTON ST, San Francisco, CA",2,3726,37.796159,12-Feb-12,"37.7961585,-122.4093167,OK",-122.409317,75,11.0,OK,11.0,2012,RED,1
968,19,"125 TRENTON ST, San Francisco, CA",NaN,3580,37.796159,NaN,"37.7961585,-122.4093167,OK",-122.409317,035,3.0,OK,3.0,2011,RM-1,1
